## Homework 4, Problem 4 Classification on real data

ECE C143A/C243A, Spring Quarter 2020, Prof. J.C. Kao, TAs J. Lee, T. Monsoor

## Background
Neural prosthetic systems can be built based on classifying neural activity related to planning. As described in class, this is analogous to mapping patterns of neural activity to keys on a keyboard.
In this problem, we will apply the results of Problems 1 and 2 to real neural data. The neural data were recorded using a 100-electrode array in premotor cortex of a macaque monkey1. The dataset can be found on CCLE as `ps4_realdata.mat`.

The following describes the data format. The `.mat` file is loaded into Python as a dictionary with two keys: `train_trial` contains the training data and `test_trial` contains the test data. Each of these contains spike trains recorded simultaneously from 97 neurons while the monkey reached 91 times along each of 8 different reaching angles.

The spike train recorded from the $i_{th}$ neuron on the $n_{th}$ trial of the $k_{th}$ reaching angle is accessed as 

`data['train_trial'][n,k][1][i,:]`

where n = 0,...,90, k = 0,...,7, and i = 0, . . . , 96.  The [1] in between [n,k] and [i,:] does not mean anything for this assignment and is simply an "artifact" of how the data is structured. A spike train is represented as a sequence of zeros and ones, where time is discretized in 1 ms steps. A zero indicates that the neuron did not spike in the 1 ms bin, whereas a one indicates that the neuron spiked once in the 1 ms bin. The structure test trial has the same format as train trial.

Each spike train is 700 ms long (and thus represented by an array of length 700).  This comprises a 200ms baseline period (before the reach target turned on), a 500ms planning period (after the reach target turned on).  Because it takes time for information about the reach target to arrive in premotor cortex (due to the time required for action potentials to propagate and for visual processing), we will ignore the first 150ms of the planning period.  *** FOR THIS PROBLEM, we will take spike counts for each neuron within a single 200ms bin starting 150ms after the reach target turns on. ***

In other words, to calculate firing rates, you will calculate it over the 200ms window: 

`data['train_trial'][n,k][1][i,350:550]`

In [1]:
import numpy as np
import numpy.matlib as npm
import matplotlib.pyplot as plt
import scipy.special
import scipy.io as sio
import math

data = sio.loadmat('ps4_realdata.mat') # load the .mat file.
NumTrainData = data['train_trial'].shape[0]
NumClass = data['train_trial'].shape[1]
NumTestData = data['test_trial'].shape[0]

# Reloading any code written in external .py files.
%load_ext autoreload
%autoreload 2

### (a) (8 points) 
Fit the ML parameters of model i) to the training data (91 × 8 observations of a length 97 array of neuron firing rates). 

To calculate the firing rates, use a single 200ms bin starting from 150ms after the target turns on.  This corresponds to using `data['train_trial'][n,k][1][i, 350:550]` to calculate all firing rates.  This corresponds to a 200ms window that turns on 150ms after the reach turns on.

Then, use these parameters to classify the test data (91 × 8 data points) according to the decision rule (1). What is the percent of test data points correctly classified?

In [2]:
##4a

# Calculate the firing rates.
TrainData = data["train_trial"]
TestData = data["test_trial"]

# Contains the firing rates for all neurons on all 8 x 91 trials
trainDataArr =  np.zeros((NumClass,NumTrainData,97)) 
testDataArr =  np.zeros((NumClass,NumTestData,97))
bin_width = 0.2 #200 ms bin

for classIX in range(NumClass):
    for trainDataIX in range(NumTrainData):   
        total_train = np.sum(data['train_trial'][trainDataIX,classIX][1][:,350:550],1)
        trainDataArr[classIX,trainDataIX,:] = total_train / bin_width
    for testDataIX in range(NumTestData):  
        total_test = np.sum(data['test_trial'][testDataIX,classIX][1][:,350:550],1)
        testDataArr[classIX,testDataIX,:] = total_test / bin_width
#====================================================#
# YOUR CODE HERE:
#   Fit the ML parameters of model i) to training data
#====================================================#
# Class Prior Calculations
trainParam1 = {} # Dictionary for Gaussian shared covariance
training_prior = float(NumTrainData) / (NumClass * NumTrainData)
trainParam1["pi"] = np.array([training_prior for i in range(NumClass)])

# Mean and Covariance Calculations
neuron_num = trainDataArr.shape[2] # Extracts number of neurons
trainmu_param1 = np.zeros((NumClass, neuron_num)) # Holds means of all data

# n holds our class specific data, trained_shared_cov our class covariances
n = np.zeros((neuron_num, NumTrainData)) # A 97 x 91 matrix for each class
trained_shared_cov = np.zeros((neuron_num, neuron_num)) # A 97 x 97 matrix

for classIX in range(NumClass): 
    for neuron in range(neuron_num):
        neuron_data = trainDataArr[classIX][:, neuron]
        n[neuron] = neuron_data
        
        neuron_mu = np.mean(neuron_data)
        trainmu_param1[classIX][neuron] = neuron_mu
        pass
    trained_shared_cov += np.cov(n)
    pass

trainParam1["mean"] = trainmu_param1
trainParam1["cov"] = trained_shared_cov / NumClass

# Gather model i) parameters
sig_inv_shared = np.linalg.inv(trainParam1["cov"])
c = -1.0 / 2 # Common constant

# Retrieve parameters for each class
w_k_params = np.zeros((NumClass, 1, neuron_num)) # Stores w_k values
w_k_consts = np.zeros((NumClass))

for classIX in range(NumClass):
    class_mu = trainParam1["mean"][classIX] # Retrieve class mean
    
    w_k = sig_inv_shared.dot(class_mu)
    w_k_params[classIX] = w_k
    
    w_const = c * ((class_mu.T).dot(sig_inv_shared)).dot(class_mu)
    w_k_consts[classIX] = w_const
    pass
#====================================================#
# END YOUR CODE
#====================================================# 

#====================================================#
# YOUR CODE HERE:
#   Classify the test data and print the accuracy
#====================================================#
# Classify test data, total of 8 x 91 trials
error_total = 0 # Keeps track of erroneously classified trials
total = NumClass * NumTestData

for classIX in range(NumClass):
    for dataIX in range(NumTestData):
        inputs = testDataArr[classIX][dataIX]
        
        # Calculate alpha(x) for all classes
        z_1 = (w_k_params[0][0].T).dot(inputs) + w_k_consts[0]
        z_2 = (w_k_params[1][0].T).dot(inputs) + w_k_consts[1]
        z_3 = (w_k_params[2][0].T).dot(inputs) + w_k_consts[2]
        z_4 = (w_k_params[3][0].T).dot(inputs) + w_k_consts[3]
        z_5 = (w_k_params[4][0].T).dot(inputs) + w_k_consts[4]
        z_6 = (w_k_params[5][0].T).dot(inputs) + w_k_consts[5]
        z_7 = (w_k_params[6][0].T).dot(inputs) + w_k_consts[6]
        z_8 = (w_k_params[7][0].T).dot(inputs) + w_k_consts[7]
        
        # Obtain the classified class number
        probs = [z_1, z_2, z_3, z_4, z_5, z_6, z_7, z_8]
        classified_num = probs.index(max(probs)) # Get the classified class
        
        # if the class is incorrectly classified, add 1 to the error number
        if (classified_num != classIX):
            error_total += 1
            pass
        pass
    pass

# Display our accuracy
msg = "This is our accuracy of classified test data points: "
accuracy = "{:.2f}".format((float(total - error_total) / total) * 100.0)
print(msg + accuracy + "%")
#====================================================#
# END YOUR CODE
#====================================================# 

This is our accuracy of classified test data points: 96.02%


#### Question:
What is the percent of test data points correctly classified?

#### Your answer: 96.02% of the test data points are correctly classified.


### (b) (6 points) 
Repeat part (a) for model ii). You `should encounter a Python error` when classifying the test data. What is this error? Why did the Python error occur? What would we need to do to correct this error?

To be concrete, the output of this cell should be a `Python error` and that's all fine.  But we want you to understand what the error is so we can fix it later.


In [3]:
##4b

#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters of model ii) to training data
#====================================================#
# Calculate class specific covariances
trainParam2 = {} # Dictionary for Gaussian specific covariance
trainParam2["pi"] = trainParam1["pi"]
trainParam2["mean"] = trainParam1["mean"]

# Specific Covariance Calculations
n1 = np.zeros((NumClass, 97, 97)) # 8 x 97 x 97 for all covariances
trained_specific_cov = np.zeros((neuron_num, NumTrainData)) # A 97 x 91 matrix

for classIX in range(NumClass):
    for neuron in range(neuron_num):
        trained_specific_cov[neuron] = trainDataArr[classIX][:, neuron]
        pass
    cov_specific = np.cov(trained_specific_cov)
    n1[classIX, :] = cov_specific
    pass

trainParam2["cov"] = n1

w_k_params = np.zeros((NumClass, 1, neuron_num))
w_k_consts = np.zeros((NumClass))

for classIX in range(NumClass):
    class_mu = trainParam2["mean"][classIX]
    sig_inv_specific = np.linalg.inv(trainParam2["cov"][classIX])

    w_k = sig_inv_specific.dot(class_mu)
    w_k_params[classIX] = w_k
    
    w_const = c * ((class_mu.T).dot(sig_inv_specific)).dot(class_mu)
    w_k_consts[classIX] = w_const
    pass
# Did not code anything after this, because this is where the error occurred
#====================================================#
# END YOUR CODE
#====================================================# 

LinAlgError: Singular matrix

#### Question:
Why did the python error occur? What would we need to do to correct this error?

#### Your answer:  The python error occurred because we were attempting to take the inversion of a not full-rank matrix (i.e. for one class it was a 97 x 97 matrix but our rank was only 90). This by definition means the matrix is not invertible. To correct this, we can remove rows that are all zeros in order to get a full rank matrix, in which case we would be able to take the determinant of thereafter. 


### (c) (8 points) 
Correct the problem from part (b) by detecting and then removing offending neurons that cause the error. Now, what is the percent of test data points correctly classified? Is it higher or lower than your answer to part (a)? Why might this be?

In [4]:
neuronsToRemove = []
#====================================================#
# YOUR CODE HERE:
#   Detect and then remove the offending neurons, so that 
#   you no longer run into the bug in part (b).
#====================================================#
# Iterate through our data, finding our erroneous neurons
for classIX in range(NumClass):
    for neuron in range(neuron_num):
        access = trainDataArr[classIX][:, neuron]
        
        # If the neuron never fires in a class, append to neuronsToRemove
        if (all(access == 0) and (neuron not in neuronsToRemove)):
            neuronsToRemove.append(neuron)
            pass
        pass
    pass
neuronsToRemove.sort() # Sort the list from least to greatest
err_n = len(neuronsToRemove) # Number of defective neurons
#====================================================#
# END YOUR CODE
#====================================================# 

#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters,classify the test data and print the accuracy
#====================================================#
# Calculate new means and covariances
new_specificov = np.zeros((NumClass, neuron_num - err_n, neuron_num - err_n))
new_specificlass = np.zeros((neuron_num - err_n, NumTrainData)) 
trainmu_param2 = np.zeros((NumClass, neuron_num - err_n))

for classIX in range(NumClass):
    counter = 0 # Index counter to account for defective neurons
    for neuron in range(neuron_num):
        if (neuron not in neuronsToRemove):
            valid_data = trainDataArr[classIX][:, neuron]
            new_specificlass[counter, :] = valid_data
            
            trainmu_param2[classIX][counter] = np.mean(valid_data)
            counter += 1
            pass
        pass
    new_specificov[classIX, :] = np.cov(new_specificlass)
    pass
trainParam2["cov"] = new_specificov
trainParam2["mean"] = trainmu_param2

# Retrieve the parameters
w_k_params = np.zeros((NumClass, 1, neuron_num - err_n))
w_k_consts = np.zeros((NumClass))

for classIX in range(NumClass):
    class_mu = trainParam2["mean"][classIX]
    sig_inv_specific = np.linalg.inv(trainParam2["cov"][classIX])
    
    w_k_params[classIX] = w_k = sig_inv_specific.dot(class_mu)
    
    w_const = c * ((class_mu.T).dot(sig_inv_specific)).dot(class_mu)
    w_const += c * np.log(np.linalg.det(trainParam2["cov"][classIX]))
    w_k_consts[classIX] = w_const
    pass

# Classify test data, total of 8 x 91 trials
error_total = 0 # Keeps track of erroneously classified trials
total = NumClass * NumTestData # Total number of trials

for classIX in range(NumClass):
    for dataIX in range(NumTestData):
        inputs = np.zeros((0))
        
        for neuron in range(neuron_num):
            if (neuron not in neuronsToRemove):
                inputs = np.append(inputs, testDataArr[classIX][dataIX][neuron])
                pass
            pass
        
        # Calculate quadratic terms
        quad1 = (inputs.T).dot(np.linalg.inv(trainParam2["cov"][0]))
        quad1 = quad1.dot(inputs)
        
        quad2 = (inputs.T).dot(np.linalg.inv(trainParam2["cov"][1]))
        quad2 = quad2.dot(inputs)
        
        quad3 = (inputs.T).dot(np.linalg.inv(trainParam2["cov"][2]))
        quad3 = quad3.dot(inputs)
        
        quad4 = (inputs.T).dot(np.linalg.inv(trainParam2["cov"][3]))
        quad4 = quad4.dot(inputs)
        
        quad5 = (inputs.T).dot(np.linalg.inv(trainParam2["cov"][4]))
        quad5 = quad5.dot(inputs)
        
        quad6 = (inputs.T).dot(np.linalg.inv(trainParam2["cov"][5]))
        quad6 = quad6.dot(inputs)
        
        quad7 = (inputs.T).dot(np.linalg.inv(trainParam2["cov"][6]))
        quad7 = quad7.dot(inputs)
        
        quad8 = (inputs.T).dot(np.linalg.inv(trainParam2["cov"][7]))
        quad8 = quad8.dot(inputs)
        
        # Calculate alpha(x) for all classes
        z_1 = (w_k_params[0][0].T).dot(inputs) + w_k_consts[0] + c * quad1
        z_2 = (w_k_params[1][0].T).dot(inputs) + w_k_consts[1] + c * quad2
        z_3 = (w_k_params[2][0].T).dot(inputs) + w_k_consts[2] + c * quad3
        z_4 = (w_k_params[3][0].T).dot(inputs) + w_k_consts[3] + c * quad4
        z_5 = (w_k_params[4][0].T).dot(inputs) + w_k_consts[4] + c * quad5
        z_6 = (w_k_params[5][0].T).dot(inputs) + w_k_consts[5] + c * quad6
        z_7 = (w_k_params[6][0].T).dot(inputs) + w_k_consts[6] + c * quad7
        z_8 = (w_k_params[7][0].T).dot(inputs) + w_k_consts[7] + c * quad8
        
        # Obtain the classified class number
        probs = [z_1, z_2, z_3, z_4, z_5, z_6, z_7, z_8]
        classified_num = probs.index(max(probs))
        
        if (classified_num != classIX):
            error_total += 1
            pass
        pass
    pass

# Display our accuracy
msg = "This is our accuracy of classified test data points: "
accuracy = "{:.2f}".format((float(total - error_total) / total) * 100.0)
print(msg + accuracy + "%")
#====================================================#
# END YOUR CODE
#====================================================# 

This is our accuracy of classified test data points: 44.09%


#### Question:
What is the percent of test data points correctly classified? Is it higher or lower than your answer to part (a)? Why might this be?

#### Your answer:  44.09% of the test data points are correctly classified. It is definitely lower than my answer to part (a). One reason for this drop in accuracy is that by removing non-firing neurons, we are reducing the size of our training set, meaning our model will be less accurate (the larger the training set, the more stable our predictions will be), thus making it more susceptible to influence from small variance neurons.

### (d) (8 points) 
Now we classify using a naive Bayes model. Repeat part (a) for model iii). Keep the convention in part (c), where offending neurons were removed from the anal- ysis.

In [5]:
##4d
#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters,classify the test data and print the accuracy
#====================================================#
trainParam3 = {} # Dictionary for Poisson distribution
trainParam3["pi"] = trainParam2["pi"]
trainParam3["mean"] = trainParam2["mean"]

poiss_error_total = 0 # Keeps track of erroneously classified trials
total = NumClass * NumTestData # Total number of trials

for classIX in range(NumClass):
    for dataIX in range(NumTestData):
        inputs = np.zeros((0))
        for neuron in range(neuron_num):
            if (neuron not in neuronsToRemove):
                inputs = np.append(inputs, testDataArr[classIX][dataIX][neuron])
                pass
            pass
        
        # Retrieve Poisson parameters
        poissmean1 = trainParam3["mean"][0]
        poissmean2 = trainParam3["mean"][1]
        poissmean3 = trainParam3["mean"][2]
        poissmean4 = trainParam3["mean"][3]
        poissmean5 = trainParam3["mean"][4]
        poissmean6 = trainParam3["mean"][5]
        poissmean7 = trainParam3["mean"][6]
        poissmean8 = trainParam3["mean"][7]

        # Calculate alpha(x) for all classes
        z_1 = inputs.dot(np.log(poissmean1))
        z_2 = inputs.dot(np.log(poissmean2))
        z_3 = inputs.dot(np.log(poissmean3))
        z_4 = inputs.dot(np.log(poissmean4))
        z_5 = inputs.dot(np.log(poissmean5))
        z_6 = inputs.dot(np.log(poissmean6))
        z_7 = inputs.dot(np.log(poissmean7))
        z_8 = inputs.dot(np.log(poissmean8))
            
        for i in range(neuron_num - err_n): # Subtract lambda_{ki}
            z_1 -= poissmean1[i]
            z_2 -= poissmean2[i]
            z_3 -= poissmean3[i]
            z_4 -= poissmean4[i]
            z_5 -= poissmean5[i]
            z_6 -= poissmean6[i]
            z_7 -= poissmean7[i]
            z_8 -= poissmean8[i]
        
        # Obtain the classified class number
        probs = [z_1, z_2, z_3, z_4, z_5, z_6, z_7, z_8]
        classified_num = probs.index(max(probs))
        
        # If the class is incorrectly classified, add 1 to the error number
        if (classified_num != classIX):
            poiss_error_total += 1
            pass
        pass
    pass

# Display our accuracy
msg = "This is our accuracy of classified test data points: "
accuracy = "{:.2f}".format((float(total - poiss_error_total) / total) * 100.0)
print(msg + accuracy + "%")
        
#====================================================#
# END YOUR CODE
#====================================================# 

This is our accuracy of classified test data points: 92.03%


#### Question:
what is the percent of test data points correctly classified? 

#### Your answer: 92.03% of the test data points are correctly classified.
